In [124]:
#Load the necessary Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
import statsmodels.api as sm
from sklearn.metrics import r2_score, mean_absolute_percentage_error
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import Ridge, Lasso
import os
from sklearn.feature_extraction.text import CountVectorizer
path=os.path.dirname(os.path.abspath("__file__"))
print(path)

C:\Users\40104157\MS Assignments\Linear Regression Assignment\Surprise Housing Assignment


In [2]:
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None)

In [8]:
def missingValuesCheck(x):
    l = []
    for i in x.columns:
        l.append(x[i].isnull().sum())

    missing = pd.DataFrame()
    missing['columns'] = x.columns
    missing['missing'] = l
    missing['pct_missing'] = round((missing.missing / x.shape[0])*100,2)
    missing = missing[missing.missing!=0]

    return missing.sort_values('pct_missing', ascending=False)

In [4]:
#Load the dataset
df = pd.read_csv(path+'/train.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [11]:
# Checking for missing values

mv_df = missingValuesCheck(df)
mv_df.head(10)

,columns,missing,pct_missing
72,PoolQC,1453,99.52
74,MiscFeature,1406,96.30
6,Alley,1369,93.77
73,Fence,1179,80.75
57,FireplaceQu,690,47.26
3,LotFrontage,259,17.74
58,GarageType,81,5.55
59,GarageYrBlt,81,5.55
60,GarageFinish,81,5.55
63,GarageQual,81,5.55


In [19]:
# Dropping all the columns that had more than 10% of missing values

drop_cols = list(mv_df[mv_df.pct_missing>10]['columns'])
drop_cols

df2 = df.drop(drop_cols, axis=1)
df2.head(2)

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500


In [27]:
# Dropping all the rows that have missing values

df3 = df2.dropna()
df3.head(2)

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500


In [30]:
# No missing values in the final data

missingValuesCheck(df3)

,columns,missing,pct_missing


In [50]:
''' Removing 'YearBuilt' as it is highly correlated with 'YearRemodAdd'. Keeping 'YearRemodAdd' makes more sense as it is the 
modelled year'''

print(df3[['YearBuilt', 'YearRemodAdd']].corr())

df3.drop('YearBuilt', axis=1, inplace=True)

              YearBuilt  YearRemodAdd
YearBuilt      1.000000      0.617156
YearRemodAdd   0.617156      1.000000


C:\Users\40104157\Anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [51]:
df_nunique = pd.DataFrame()
cols = []
values = []
for i in df3.columns:
    cols.append(i)
    values.append(df3[i].nunique())

df_nunique['Columns'] = cols
df_nunique['Unique Values'] = values
df_nunique.head()

df_nunique.sort_values('Unique Values', ascending=True)

,Columns,Unique Values
38,CentralAir,2
4,Street,2
7,Utilities,2
49,KitchenAbvGr,3
61,PavedDrive,3
45,BsmtHalfBath,3
44,BsmtFullBath,3
47,HalfBath,3
9,LandSlope,3
56,GarageFinish,3


In [54]:
# We will now use one hot encoding for the categorical variables

ohe_cols = list(df_nunique[df_nunique['Unique Values']<10]['Columns'])
dummy_vars_df = pd.get_dummies(df3[ohe_cols], drop_first=True)

dummy_vars_df.head()

,OverallQual,OverallCond,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,Fireplaces,GarageCars,PoolArea,YrSold,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_NoSeWa,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Mod,LandSlope_Sev,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Fa,ExterCond_Gd,ExterCond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Stone,Foundation_Wood,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasW,Heating_Grav,Heating_OthW,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_P,PavedDrive_Y,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,7,5,1,0,2,1,3,1,0,2,0,2008,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0
1,6,8,0,1,2,0,3,1,1,2,0,2007,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0
2,7,5,1,0,2,1,3,1,1,2,0,2008,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0
3,7,5,1,0,1,0,3,1,1,3,0,2006,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
4,8,5,1,0,2,1,4,1,1,3,0,2008,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0


In [55]:
# Dropping the encoded columns and adding the dummies for the same

df4 = df3.drop(ohe_cols, axis=1)

df5 = pd.concat([df4, dummy_vars_df], axis=1)

df5.head()

,Id,MSSubClass,LotArea,Neighborhood,YearRemodAdd,Exterior1st,Exterior2nd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,TotRmsAbvGrd,GarageYrBlt,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,MiscVal,MoSold,SalePrice,OverallQual,OverallCond,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,Fireplaces,GarageCars,PoolArea,YrSold,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_NoSeWa,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Mod,LandSlope_Sev,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Fa,ExterCond_Gd,ExterCond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Stone,Foundation_Wood,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasW,Heating_Grav,Heating_OthW,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_P,PavedDrive_Y,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,8450,CollgCr,2003,VinylSd,VinylSd,196.0,706,0,150,856,856,854,0,1710,8,2003.0,548,0,61,0,0,0,0,2,208500,7,5,1,0,2,1,3,1,0,2,0,2008,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0
1,2,20,9600,Veenker,1976,MetalSd,MetalSd,0.0,978,0,284,1262,1262,0,0,1262,6,1976.0,460,298,0,0,0,0,0,5,181500,6,8,0,1,2,0,3,1,1,2,0,2007,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0
2,3,60,11250,CollgCr,2002,VinylSd,VinylSd,162.0,486,0,434,920,920,866,0,1786,6,2001.0,608,0,42,0,0,0,0,9,223500,7,5,1,0,2,1,3,1,1,2,0,2008,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0
3,4,70,9550,Crawfor,1970,Wd Sdng,Wd Shng,0.0,216,0,540,756,961,756,0,1717,7,1998.0,642,0,35,272,0,0,0,2,140000,7,5,1,0,1,0,3,1,1,3,0,2006,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0

In [60]:
df5[['GarageYrBlt', 'YrSold']] = df5[['GarageYrBlt', 'YrSold']].astype('object')

In [66]:
cols = ['Neighborhood', 'Exterior1st', 'Exterior2nd', 'GarageYrBlt', 'YrSold', 'GarageCars', 'BedroomAbvGr', 'FullBath', 'MSSubClass']
dummy_vars_df_2 = pd.get_dummies(df5[cols], drop_first=True)
dummy_vars_df_2.head()

,GarageCars,BedroomAbvGr,FullBath,MSSubClass,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,GarageYrBlt_1906.0,GarageYrBlt_1908.0,GarageYrBlt_1910.0,GarageYrBlt_1914.0,GarageYrBlt_1915.0,GarageYrBlt_1916.0,GarageYrBlt_1918.0,GarageYrBlt_1920.0,GarageYrBlt_1921.0,GarageYrBlt_1922.0,GarageYrBlt_1923.0,GarageYrBlt_1924.0,GarageYrBlt_1925.0,GarageYrBlt_1926.0,GarageYrBlt_1927.0,GarageYrBlt_1928.0,GarageYrBlt_1929.0,GarageYrBlt_1930.0,GarageYrBlt_1931.0,GarageYrBlt_1932.0,GarageYrBlt_1933.0,GarageYrBlt_1934.0,GarageYrBlt_1935.0,GarageYrBlt_1936.0,GarageYrBlt_1937.0,GarageYrBlt_1938.0,GarageYrBlt_1939.0,GarageYrBlt_1940.0,GarageYrBlt_1941.0,GarageYrBlt_1942.0,GarageYrBlt_1945.0,GarageYrBlt_1946.0,GarageYrBlt_1947.0,GarageYrBlt_1948.0,GarageYrBlt_1949.0,GarageYrBlt_1950.0,GarageYrBlt_1951.0,GarageYrBlt_1952.0,GarageYrBlt_1953.0,GarageYrBlt_1954.0,GarageYrBlt_1955.0,GarageYrBlt_1956.0,GarageYrBlt_1957.0,GarageYrBlt_1958.0,GarageYrBlt_1959.0,GarageYrBlt_1960.0,GarageYrBlt_1961.0,GarageYrBlt_1962.0,GarageYrBlt_1963.0,GarageYrBlt_1964.0,GarageYrBlt_1965.0,GarageYrBlt_1966.0,GarageYrBlt_1967.0,GarageYrBlt_1968.0,GarageYrBlt_1969.0,GarageYrBlt_1970.0,GarageYrBlt_1971.0,GarageYrBlt_1972.0,GarageYrBlt_1973.0,GarageYrBlt_1974.0,GarageYrBlt_1975.0,GarageYrBlt_1976.0,GarageYrBlt_1977.0,GarageYrBlt_1978.0,GarageYrBlt_1979.0,GarageYrBlt_1980.0,GarageYrBlt_1981.0,GarageYrBlt_1982.0,GarageYrBlt_1983.0,GarageYrBlt_1984.0,GarageYrBlt_1985.0,GarageYrBlt_1986.0,GarageYrBlt_1987.0,GarageYrBlt_1988.0,GarageYrBlt_1989.0,GarageYrBlt_1990.0,GarageYrBlt_1991.0,GarageYrBlt_1992.0,GarageYrBlt_1993.0,GarageYrBlt_1994.0,GarageYrBlt_1995.0,GarageYrBlt_1996.0,GarageYrBlt_1997.0,GarageYrBlt_1998.0,GarageYrBlt_1999.0,GarageYrBlt_2000.0,GarageYrBlt_2001.0,GarageYrBlt_2002.0,GarageYrBlt_2003.0,GarageYrBlt_2004.0,GarageYrBlt_2005.0,GarageYrBlt_2006.0,GarageYrBlt_2007.0,GarageYrBlt_2008.0,GarageYrBlt_2009.0,GarageYrBlt_2010.0,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010
0,2,3,2,60,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
1,2,3,2,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,2,3,2,60,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
3,3,3,1,70,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,

In [67]:
# Dropping the encoded columns and adding the dummies for the same

df6 = df5.drop(cols, axis=1)

df7 = pd.concat([df6, dummy_vars_df_2], axis=1)

df7.head()

,Id,LotArea,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,TotRmsAbvGrd,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,MiscVal,MoSold,SalePrice,OverallQual,OverallCond,BsmtFullBath,BsmtHalfBath,HalfBath,KitchenAbvGr,Fireplaces,PoolArea,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_NoSeWa,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Mod,LandSlope_Sev,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Fa,ExterCond_Gd,ExterCond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Stone,Foundation_Wood,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasW,Heating_Grav,Heating_OthW,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_P,PavedDrive_Y,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,GarageCars,BedroomAbvGr,FullBath,MSSubClass,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,GarageYrBlt_1906.0,GarageYrBlt_1908.0,GarageYrBlt_1910.0,GarageYrBlt_1914.0,GarageYrBlt_1915.0,GarageYrBlt_1916.0,GarageYrBlt_1918.0,GarageYrBlt_1920.0,GarageYrBlt_1921.0,GarageYrBlt_1922.0,GarageYrBlt_1923.0,GarageYrBlt_1924.0,GarageYrBlt_1925.0,GarageYrBlt_1926.0,GarageYrBlt_1927.0,GarageYrBlt_1928.0,GarageYrBlt_1929.0,GarageYrBlt_1930.0,GarageYrBlt_1931.0,GarageYrBlt_1932.0,GarageYrBlt_1933.0,GarageYrBlt_1934.0,GarageYrBlt_1935

In [68]:
# Splitting the data into train and test

df_train, df_test = train_test_split(df7, test_size=0.3, random_state=1)
print(df_train.shape)
print(df_test.shape)

(936, 325)
(402, 325)


In [69]:
# Rescalling function using min_max scaler
def min_max_scaler(data, cols):

    scaler = MinMaxScaler()
    data[cols] = scaler.fit_transform(data[cols])
    return data

In [71]:
scaling_cols = ['LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 
                'LowQualFinSF', 'GrLivArea', 'TotRmsAbvGrd', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 
               '3SsnPorch', 'ScreenPorch', 'MiscVal']

In [72]:
df_train_scaled = min_max_scaler(df_train, scaling_cols)
df_train_scaled.head()

C:\Users\40104157\Anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


,Id,LotArea,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,TotRmsAbvGrd,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,MiscVal,MoSold,SalePrice,OverallQual,OverallCond,BsmtFullBath,BsmtHalfBath,HalfBath,KitchenAbvGr,Fireplaces,PoolArea,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_NoSeWa,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Mod,LandSlope_Sev,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Fa,ExterCond_Gd,ExterCond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Stone,Foundation_Wood,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasW,Heating_Grav,Heating_OthW,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_P,PavedDrive_Y,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,GarageCars,BedroomAbvGr,FullBath,MSSubClass,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,GarageYrBlt_1906.0,GarageYrBlt_1908.0,GarageYrBlt_1910.0,GarageYrBlt_1914.0,GarageYrBlt_1915.0,GarageYrBlt_1916.0,GarageYrBlt_1918.0,GarageYrBlt_1920.0,GarageYrBlt_1921.0,GarageYrBlt_1922.0,GarageYrBlt_1923.0,GarageYrBlt_1924.0,GarageYrBlt_1925.0,GarageYrBlt_1926.0,GarageYrBlt_1927.0,GarageYrBlt_1928.0,GarageYrBlt_1929.0,GarageYrBlt_1930.0,GarageYrBlt_1931.0,GarageYrBlt_1932.0,GarageYrBlt_1933.0,GarageYrBlt_1934.0,GarageYrBlt_1935

In [73]:
df_test_scaled = min_max_scaler(df_test, scaling_cols)
df_test_scaled.head()

C:\Users\40104157\Anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


,Id,LotArea,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,TotRmsAbvGrd,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,MiscVal,MoSold,SalePrice,OverallQual,OverallCond,BsmtFullBath,BsmtHalfBath,HalfBath,KitchenAbvGr,Fireplaces,PoolArea,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_NoSeWa,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Mod,LandSlope_Sev,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Fa,ExterCond_Gd,ExterCond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Stone,Foundation_Wood,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasW,Heating_Grav,Heating_OthW,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_P,PavedDrive_Y,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,GarageCars,BedroomAbvGr,FullBath,MSSubClass,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,GarageYrBlt_1906.0,GarageYrBlt_1908.0,GarageYrBlt_1910.0,GarageYrBlt_1914.0,GarageYrBlt_1915.0,GarageYrBlt_1916.0,GarageYrBlt_1918.0,GarageYrBlt_1920.0,GarageYrBlt_1921.0,GarageYrBlt_1922.0,GarageYrBlt_1923.0,GarageYrBlt_1924.0,GarageYrBlt_1925.0,GarageYrBlt_1926.0,GarageYrBlt_1927.0,GarageYrBlt_1928.0,GarageYrBlt_1929.0,GarageYrBlt_1930.0,GarageYrBlt_1931.0,GarageYrBlt_1932.0,GarageYrBlt_1933.0,GarageYrBlt_1934.0,GarageYrBlt_1935

In [75]:
model_cols = list(df_train_scaled.columns)
model_cols.remove('Id')
model_cols.remove('SalePrice')

target_col = ['SalePrice']

In [76]:
xtrain_sm = sm.add_constant(df_train_scaled[model_cols])

lr = sm.OLS(df_train_scaled[target_col], xtrain_sm)

lr_model = lr.fit()

lr_model.summary()

C:\Users\40104157\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.952
Model:                            OLS   Adj. R-squared:                  0.929
Method:                 Least Squares   F-statistic:                     40.98
Date:                Sat, 14 May 2022   Prob (F-statistic):          4.43e-293
Time:                        20:41:20   Log-Likelihood:                -10455.
No. Observations:                 936   AIC:                         2.152e+04
Df Residuals:                     630   BIC:                         2.300e+04
Df Model:                         305                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -2.394e+05   1.52e+05     -1.570      0.117   -5.39e+05       6e+04
LotArea                1.931e+05   2.76e+04      7.005      0.000    1.39e+05    2.47e+05
YearRemodAdd            124.9728     79.656      1.569      0.117     -31.450     281.396
MasVnrArea             2.603e+04   1.03e+04      2.534      0.012    5860.419    4.62e+04
BsmtFinSF1             4.809e+04   7547.173      6.373      0.000    3.33e+04    6.29e+04
BsmtFinSF2             1.779e+04   1.41e+04      1.261      0.208   -9925.054    4.55e+04
BsmtUnfSF              2.467e+04   8401.342      2.936      0.003    8171.336    4.12e+04
TotalBsmtSF             7.34e+04   1.26e+04      5.821      0.000    4.86e+04    9.82e+04
1stFlrSF               4.244e+04   1.33e+04      3.194      0.001    1.63e+04    6.85e+04
2ndFlrSF               8.109e+04   1.15e+04      7.024      0.000    5.84e+04    1.04e+05
LowQualFinSF           1.808e+04   1.51e+04      1.196      0.232   -1.16e+04    4.78e+04
GrLivArea               8.66e+04      1e+04      8.646      0.000    6.69e+04    1.06e+05
TotRmsAbvGrd           9454.7538   1.11e+04      0.853      0.394   -1.23e+04    3.12e+04
GarageArea             1.595e+04   1.15e+04      1.387      0.166   -6634.676    3.85e+04
WoodDeckSF             1.446e+04   6313.929      2.290      0.022    2062.163    2.69e+04
OpenPorchSF            1.712e+04   8566.024      1.999      0.046     300.821    3.39e+04
EnclosedPorch          1453.1132   9108.776      0.160      0.873   -1.64e+04    1.93e+04
3SsnPorch              2.269e+04   1.35e+04      1.675      0.094   -3913.173    4.93e+04
ScreenPorch             2.08e+04   7261.372      2.865      0.004    6543.328    3.51e+04
MiscVal                4524.8895   2.29e+04      0.198      0.843   -4.05e+04    4.95e+04
MoSold                 -235.5705    319.885     -0.736      0.462    -863.741     392.600
OverallQual            5682.3356   1405.704      4.042      0.000    2921.903    8442.768
OverallCond            3869.9162   1224.824      3.160      0.002    1464.685    6275.147
BsmtFullBath           2560.7150   2581.779      0.992      0.322   -2509.218    7630.648
BsmtHalfBath           -598.8179   4110.652     -0.146      0.884   -8671.056    7473.420
HalfBath              -1698.3773   2793.919     -0.608      0.543   -7184.898    3788.143
KitchenAbvGr          -2.863e+04   1.19e+04     -2.406      0.016    -5.2e+04   -5264.094
Fireplaces              -66.3998   1736.667     -0.038      0.970   -3476.756    3343.957
PoolArea                 10.8029     21.408      0.505      0.614     -31.238      52.843
MSZoning_FV            4.624e+04    2.3e+04      2.014      0.044    1145.010    9.13e+04
MSZoning_RH            3.891e+04   2.36e+04      1.649      0.100   -7425.370    8.52e+04
MSZoning_RL            

In [78]:
y_train_pred = lr_model.predict(xtrain_sm)

In [79]:
xtest_sm = sm.add_constant(df_test_scaled[model_cols])
y_test_pred = lr_model.predict(xtest_sm)

In [80]:
# We can clearly see that the model is overfitting

print('Train r2: ', r2_score(df_train_scaled[target_col], y_train_pred))

print('Test r2: ', r2_score(df_test_scaled[target_col], y_test_pred))

Train r2:  0.9520120391668366
Test r2:  0.5347107184155626


In [91]:
def vif_function(data, cols):
    vif_data = pd.DataFrame()
    vif_data["feature"] = cols
    vif_data["VIF"] = [variance_inflation_factor(data[cols].values, i) for i in range(len(data[cols].columns))]
    return vif_data.sort_values('VIF', ascending=False)

In [93]:
vif_df = vif_function(df_train_scaled, model_cols)

C:\Users\40104157\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\40104157\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


In [94]:
vif_df.head()

,feature,VIF
78,RoofMatl_Membran,inf
10,GrLivArea,inf
82,RoofMatl_WdShake,inf
83,RoofMatl_WdShngl,inf
77,RoofMatl_CompShg,inf


In [95]:
vif_drop_cols = list(vif_df[vif_df.VIF>=10]['feature'])
vif_drop_cols

['RoofMatl_Membran',
 'GrLivArea',
 'RoofMatl_WdShake',
 'RoofMatl_WdShngl',
 'RoofMatl_CompShg',
 'RoofMatl_Roll',
 'RoofMatl_Metal',
 'BsmtCond_Po',
 'GarageQual_Po',
 'RoofMatl_Tar&Grv',
 'LowQualFinSF',
 '2ndFlrSF',
 '1stFlrSF',
 'TotalBsmtSF',
 'BsmtUnfSF',
 'Electrical_Mix',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'RoofStyle_Gable',
 'RoofStyle_Hip',
 'Exterior1st_VinylSd',
 'Exterior2nd_VinylSd',
 'ExterCond_TA',
 'ExterCond_Gd',
 'GarageYrBlt_2005.0',
 'MSZoning_RL',
 'GarageYrBlt_2006.0',
 'GarageYrBlt_2003.0',
 'MSSubClass',
 'Exterior1st_MetalSd',
 'SaleType_New',
 'SaleCondition_Partial',
 'Exterior2nd_MetalSd',
 'GarageYrBlt_2004.0',
 'GarageCond_TA',
 'MSZoning_RM',
 'Exterior1st_HdBoard',
 'GarageQual_TA',
 'Exterior2nd_HdBoard',
 'GarageYrBlt_2007.0',
 'Exterior1st_Wd Sdng',
 'Exterior2nd_Wd Sdng',
 'GarageCond_Fa',
 'GarageQual_Fa',
 'GarageYrBlt_1977.0',
 'GarageYrBlt_2008.0',
 'GarageYrBlt_1976.0',
 'GarageYrBlt_1999.0',
 'GarageType_Attchd',
 'GarageYrBlt_1998.0',
 'BldgType

In [96]:
# Dropping features that have significantly higher VIF

df_train_scaled.drop(vif_drop_cols, axis=1, inplace=True)
df_test_scaled.drop(vif_drop_cols, axis=1, inplace=True)

C:\Users\40104157\Anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [97]:
model_cols = list(df_train_scaled.columns)
model_cols.remove('Id')
model_cols.remove('SalePrice')

target_col = ['SalePrice']

In [98]:
xtrain_sm = sm.add_constant(df_train_scaled[model_cols])

lr = sm.OLS(df_train_scaled[target_col], xtrain_sm)

lr_model = lr.fit()

lr_model.summary()

C:\Users\40104157\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.894
Model:                            OLS   Adj. R-squared:                  0.874
Method:                 Least Squares   F-statistic:                     44.11
Date:                Sat, 14 May 2022   Prob (F-statistic):          1.82e-299
Time:                        21:13:48   Log-Likelihood:                -10826.
No. Observations:                 936   AIC:                         2.195e+04
Df Residuals:                     785   BIC:                         2.269e+04
Df Model:                         150                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -7.348e+05   1.63e+05     -4.499      0.000   -1.06e+06   -4.14e+05
LotArea                2.296e+05   2.86e+04      8.022      0.000    1.73e+05    2.86e+05
YearRemodAdd            331.3440     81.460      4.068      0.000     171.439     491.249
MasVnrArea             2.683e+04   9067.428      2.959      0.003    9028.621    4.46e+04
TotRmsAbvGrd           8.946e+04   1.07e+04      8.331      0.000    6.84e+04    1.11e+05
GarageArea             6.505e+04   1.25e+04      5.220      0.000    4.06e+04    8.95e+04
WoodDeckSF              2.59e+04   7386.263      3.506      0.000    1.14e+04    4.04e+04
OpenPorchSF            4.407e+04   9443.514      4.667      0.000    2.55e+04    6.26e+04
EnclosedPorch           106.3503   1.01e+04      0.011      0.992   -1.97e+04    1.99e+04
3SsnPorch              1.935e+04   1.58e+04      1.225      0.221   -1.16e+04    5.03e+04
ScreenPorch            2.758e+04   8398.866      3.284      0.001    1.11e+04    4.41e+04
MiscVal                -1.31e+04   2.79e+04     -0.470      0.639   -6.79e+04    4.17e+04
MoSold                 -521.6171    381.854     -1.366      0.172   -1271.194     227.960
OverallQual            1.759e+04   1362.499     12.906      0.000    1.49e+04    2.03e+04
OverallCond            2117.5183   1192.030      1.776      0.076    -222.426    4457.463
BsmtFullBath           7867.2546   2677.460      2.938      0.003    2611.425    1.31e+04
BsmtHalfBath            745.0225   4641.132      0.161      0.873   -8365.476    9855.521
HalfBath               5133.5874   2383.075      2.154      0.032     455.633    9811.542
KitchenAbvGr          -3.055e+04   1.07e+04     -2.849      0.004   -5.16e+04   -9504.292
Fireplaces             4670.4069   1917.998      2.435      0.015     905.395    8435.419
PoolArea                 21.1219     24.891      0.849      0.396     -27.738      69.982
MSZoning_RH           -4742.0255   1.31e+04     -0.363      0.717   -3.04e+04    2.09e+04
Street_Pave            1.023e+05   2.23e+04      4.592      0.000    5.86e+04    1.46e+05
LotShape_IR2           3173.3133   6472.558      0.490      0.624   -9532.256    1.59e+04
LotShape_IR3           1.662e+04   1.43e+04      1.166      0.244   -1.14e+04    4.46e+04
LotShape_Reg          -1707.9102   2318.695     -0.737      0.462   -6259.487    2843.667
LandContour_HLS       -8296.5244   8255.465     -1.005      0.315   -2.45e+04    7908.876
LandContour_Low       -2.259e+04   9776.940     -2.311      0.021   -4.18e+04   -3398.235
LandContour_Lvl       -2184.4150   6152.051     -0.355      0.723   -1.43e+04    9892.003
Utilities_NoSeWa      -6.061e+04   3.32e+04     -1.827      0.068   -1.26e+05    4519.527
LotConfig_CulDSac      1.534e+04   4884.647      3.141      0.002    5752.099    2.49e+04
LotConfig_FR2         -

In [99]:
y_train_pred = lr_model.predict(xtrain_sm)

xtest_sm = sm.add_constant(df_test_scaled[model_cols])
y_test_pred = lr_model.predict(xtest_sm)

C:\Users\40104157\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [100]:
# We can clearly see that the model is overfitting

print('Train r2: ', r2_score(df_train_scaled[target_col], y_train_pred))

print('Test r2: ', r2_score(df_test_scaled[target_col], y_test_pred))

Train r2:  0.8939357180551005
Test r2:  0.46847784293437367


In [101]:
vif_df = vif_function(df_train_scaled, model_cols)
vif_df

C:\Users\40104157\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1717: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.uncentered_tss


,feature,VIF
1,YearRemodAdd,1186.235351
21,Street_Pave,580.646869
17,KitchenAbvGr,145.821058
12,OverallQual,90.931883
81,CentralAir_Y,56.278234
109,GarageCars,55.037817
13,OverallCond,52.477615
61,BsmtCond_TA,46.113401
97,PavedDrive_Y,43.831823
27,LandContour_Lvl,41.013377


In [102]:
vif_drop_cols = list(vif_df[vif_df.VIF>=10]['feature'])
vif_drop_cols

['YearRemodAdd',
 'Street_Pave',
 'KitchenAbvGr',
 'OverallQual',
 'CentralAir_Y',
 'GarageCars',
 'OverallCond',
 'BsmtCond_TA',
 'PavedDrive_Y',
 'LandContour_Lvl',
 'BedroomAbvGr',
 'FullBath',
 'SaleType_WD',
 'TotRmsAbvGrd',
 'Electrical_SBrkr',
 'SaleCondition_Normal',
 'GarageArea']

In [103]:
# Dropping features that have significantly higher VIF

df_train_scaled.drop(vif_drop_cols, axis=1, inplace=True)
df_test_scaled.drop(vif_drop_cols, axis=1, inplace=True)

C:\Users\40104157\Anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [104]:
model_cols = list(df_train_scaled.columns)
model_cols.remove('Id')
model_cols.remove('SalePrice')

target_col = ['SalePrice']

In [105]:
xtrain_sm = sm.add_constant(df_train_scaled[model_cols])

lr = sm.OLS(df_train_scaled[target_col], xtrain_sm)

lr_model = lr.fit()

lr_model.summary()

C:\Users\40104157\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.768
Model:                            OLS   Adj. R-squared:                  0.729
Method:                 Least Squares   F-statistic:                     19.93
Date:                Sat, 14 May 2022   Prob (F-statistic):          1.53e-181
Time:                        21:16:57   Log-Likelihood:                -11193.
No. Observations:                 936   AIC:                         2.265e+04
Df Residuals:                     802   BIC:                         2.330e+04
Df Model:                         133                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   1.67e+05   9555.078     17.477      0.000    1.48e+05    1.86e+05
LotArea                2.695e+05   3.89e+04      6.923      0.000    1.93e+05    3.46e+05
MasVnrArea             8.144e+04   1.29e+04      6.333      0.000    5.62e+04    1.07e+05
WoodDeckSF             3.721e+04   1.06e+04      3.505      0.000    1.64e+04    5.81e+04
OpenPorchSF            8.966e+04   1.34e+04      6.676      0.000    6.33e+04    1.16e+05
EnclosedPorch           1.48e+04   1.46e+04      1.017      0.309   -1.38e+04    4.34e+04
3SsnPorch              3.597e+04    2.3e+04      1.564      0.118   -9169.412    8.11e+04
ScreenPorch            3.458e+04    1.2e+04      2.876      0.004     1.1e+04    5.82e+04
MiscVal               -1.877e+04   4.04e+04     -0.465      0.642    -9.8e+04    6.05e+04
MoSold                 -362.2345    552.507     -0.656      0.512   -1446.766     722.297
BsmtFullBath            645.6139   3800.006      0.170      0.865   -6813.519    8104.746
BsmtHalfBath          -3614.8371   6639.353     -0.544      0.586   -1.66e+04    9417.724
HalfBath               1.489e+04   3141.277      4.739      0.000    8720.606    2.11e+04
Fireplaces             1.511e+04   2648.723      5.703      0.000    9907.140    2.03e+04
PoolArea                 18.2259     36.066      0.505      0.613     -52.569      89.021
MSZoning_RH            1.649e+04   1.82e+04      0.904      0.366   -1.93e+04    5.23e+04
LotShape_IR2           6561.5679   9401.413      0.698      0.485   -1.19e+04     2.5e+04
LotShape_IR3           3778.2230   2.06e+04      0.183      0.855   -3.67e+04    4.43e+04
LotShape_Reg          -2217.1513   3375.335     -0.657      0.511   -8842.686    4408.383
LandContour_HLS        1.054e+04   8980.376      1.174      0.241   -7089.105    2.82e+04
LandContour_Low       -1.638e+04   1.19e+04     -1.379      0.168   -3.97e+04    6940.271
Utilities_NoSeWa      -1.061e+05   4.77e+04     -2.226      0.026      -2e+05   -1.25e+04
LotConfig_CulDSac      2.323e+04   7106.524      3.268      0.001    9275.846    3.72e+04
LotConfig_FR2         -1.331e+04   9050.875     -1.471      0.142   -3.11e+04    4454.504
LotConfig_FR3          1.022e+04   2.96e+04      0.345      0.730   -4.79e+04    6.83e+04
LotConfig_Inside       2490.5512   3946.126      0.631      0.528   -5255.404    1.02e+04
LandSlope_Mod         -3598.7324   7948.515     -0.453      0.651   -1.92e+04     1.2e+04
LandSlope_Sev         -8.512e+04   1.88e+04     -4.524      0.000   -1.22e+05   -4.82e+04
Condition1_Feedr      -1.225e+04   7688.759     -1.593      0.111   -2.73e+04    2840.775
Condition1_PosA        5564.8601   1.52e+04      0.366      0.714   -2.43e+04    3.54e+04
Condition1_PosN        1.418e+04   1.23e+04      1.149      0.251      -1e+04    3.84e+04
Condition1_RRAe       -

In [106]:
y_train_pred = lr_model.predict(xtrain_sm)

xtest_sm = sm.add_constant(df_test_scaled[model_cols])
y_test_pred = lr_model.predict(xtest_sm)

C:\Users\40104157\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [107]:
# We can clearly see that the model is overfitting

print('Train r2: ', r2_score(df_train_scaled[target_col], y_train_pred))

print('Test r2: ', r2_score(df_test_scaled[target_col], y_test_pred))

Train r2:  0.7677325009140968
Test r2:  0.3067865612631515


In [ ]:
# Using Regularization

In [118]:
# Ridge - Auto Hyper parameter tuning

params = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0,
                   2.0, 3.0, 4.0, 5.0, 6.0, 7.0 , 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000]}

ridge = Ridge()
folds = 5
model_cv = GridSearchCV(estimator=ridge,
                       param_grid=params,
                       scoring='neg_mean_absolute_error',
                       cv = folds,
                       return_train_score=True,
                       verbose=1)
model_cv.fit(df_train_scaled[model_cols], df_train_scaled[target_col])

Fitting 5 folds for each of 28 candidates, totalling 140 fits


GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.2, 0.3,
                                   0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0,
                                   4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50,
                                   100, 500, 1000]},
             return_train_score=True, scoring='neg_mean_absolute_error',
             verbose=1)

In [119]:
model_cv.best_params_

{'alpha': 5.0}

In [139]:
metrics = []

In [140]:
ridge = Ridge(alpha=5)
ridge.fit(df_train_scaled[model_cols], df_train_scaled[target_col])
y_train_pred = ridge.predict(df_train_scaled[model_cols])
y_test_pred = ridge.predict(df_test_scaled[model_cols])

metrics.append(round(r2_score(df_train_scaled[target_col], y_train_pred),2))
metrics.append(round(mean_absolute_percentage_error(df_train_scaled[target_col], y_train_pred)*100,2))
metrics.append(round(r2_score(df_test_scaled[target_col], y_test_pred),2))
metrics.append(round(mean_absolute_percentage_error(df_test_scaled[target_col], y_test_pred)*100,2))

In [141]:
results = pd.DataFrame()
metric_cols = ['Train_R2', 'Train_MAPE', 'Test_R2', 'Test_MAPE']
results['Metric'] = metric_cols
results['Ridge'] = metrics

In [142]:
results

,Metric,Ridge
0,Train_R2,0.74
1,Train_MAPE,15.99
2,Test_R2,0.60
3,Test_MAPE,19.61


In [143]:
# Lasso - Auto Hyper parameter tuning

lasso = Lasso()
folds = 5
model_cv = GridSearchCV(estimator=lasso,
                       param_grid=params,
                       scoring='neg_mean_absolute_error',
                       cv = folds,
                       return_train_score=True,
                       verbose=1)
model_cv.fit(df_train_scaled[model_cols], df_train_scaled[target_col])

Fitting 5 folds for each of 28 candidates, totalling 140 fits


GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.2, 0.3,
                                   0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0,
                                   4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50,
                                   100, 500, 1000]},
             return_train_score=True, scoring='neg_mean_absolute_error',
             verbose=1)

In [144]:
model_cv.best_params_

{'alpha': 100}

In [145]:
metrics_lasso = []

lasso = Lasso(alpha=100)
lasso.fit(df_train_scaled[model_cols], df_train_scaled[target_col])
y_train_pred = lasso.predict(df_train_scaled[model_cols])
y_test_pred = lasso.predict(df_test_scaled[model_cols])


metrics_lasso.append(round(r2_score(df_train_scaled[target_col], y_train_pred),2))
metrics_lasso.append(round(mean_absolute_percentage_error(df_train_scaled[target_col], y_train_pred)*100,2))
metrics_lasso.append(round(r2_score(df_test_scaled[target_col], y_test_pred),2))
metrics_lasso.append(round(mean_absolute_percentage_error(df_test_scaled[target_col], y_test_pred)*100,2))

In [146]:
results['Lasso'] = metrics_lasso

In [147]:
results

,Metric,Ridge,Lasso
0,Train_R2,0.74,0.74
1,Train_MAPE,15.99,15.90
2,Test_R2,0.60,0.50
3,Test_MAPE,19.61,22.59


In [ ]:
'''
From the above results table we can see that Ridge has a comparable R2 and MAPE for both Train and Test as compared to Lasso.
Hence, we will be going forward with the Ridge Regression Regularization model
'''